<a href="https://colab.research.google.com/github/reznimat/ppzd_sp/blob/main/semestralni_prace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
import csv
import pandas as pd
import math as mt

url_sales = 'https://raw.githubusercontent.com/reznimat/ppzd_sp/main/Online_Sales.csv'
df_sales = pd.read_csv(url_sales)

url_coupons = 'https://raw.githubusercontent.com/reznimat/ppzd_sp/main/Discount_Coupon.csv'
df_coupons = pd.read_csv(url_coupons,sep=";")

ApparelSales = 0
AvgOrderPrice = 0

#print(sales)



# Počet a výpis distinktních produktových kategorií
uniq = set()
for i, row in df_sales.iterrows():
  uniq.add(row["Product_Category"])

#print("Produktových kategorií: "+ str(len(uniq)) )  # vypíše: Států/lokací:32
#print(uniq)  # vypíše např.: {'Romania', 'Luxembourg', 'France', 'Czechia',...}

for i, row in df_sales.iterrows():
  if row.Product_Category == 'Apparel':
    ApparelSales += row.Quantity*row.Avg_Price

print(df_coupons)

    Month      Product_Category Coupon_Code  Discount_pct
0     Jan               Apparel      SALE10            10
1     Feb               Apparel      SALE20            20
2     Mar               Apparel      SALE30            30
3     Jan              Nest-USA      ELEC10            10
4     Feb              Nest-USA      ELEC20            20
..    ...                   ...         ...           ...
199   Nov  Notebooks & Journals        NJ20            20
200   Dec  Notebooks & Journals        NJ30            30
201   Oct               Android       AND10            10
202   Nov               Android       AND20            20
203   Dec               Android       AND30            30

[204 rows x 4 columns]
